# IMAGE PREPROCESSING TECHNIQUES

In [2]:
#%%sh
# pip install -q pip --upgrade
# pip install -q --upgrade opencv-python
#pip install --user --upgrade tensorflow
#pip install -q --upgrade pillow

In [13]:
import cv2
import boto3
import os
import PIL
from io import BytesIO
from PIL import Image
import numpy as np
import tensorflow as tf

In [14]:
print(f"OpenCV Version: {cv2.__version__}") 
print(f"Pillow Version: {PIL.__version__}")

OpenCV Version: 4.1.2
Pillow Version: 7.0.0


## Pillow Method

In [15]:
S3 = boto3.client('s3')

filename = 'public/Sofy Soul.png'
image_object = S3.get_object(Bucket='pornilarity-bucket170933-production', Key=filename)
file_content = image_object["Body"].read()

dataBytesIO = BytesIO(file_content)
pillow_array = np.frombuffer(dataBytesIO.read(), np.uint8)
pillow_image = Image.open(dataBytesIO)
pillow_image_tensor = np.array(pillow_image)
pillow_gray = pillow_image.convert('L')
pillow_image = pillow_gray.resize((100,100))
pillow_image = np.array(pillow_image).reshape(-1, 100, 100, 1) # reshapes the image into accepted dimensions (4D tensor)
pillow_image = pillow_image.astype('float32') # change data type of image to float32
pillow_image = pillow_image / 255.0

## OpenCV Method

In [12]:
S3 = boto3.client('s3')

filename = 'public/Sofy Soul.png'
image_object = S3.get_object(Bucket='pornilarity-bucket170933-production', Key=filename)
file_content = image_object["Body"].read()

np_array = np.frombuffer(file_content, np.uint8)
cv2_image_tensor = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
cv2_gray_image = cv2.cvtColor(image_tensor, cv2.COLOR_BGR2GRAY)

cv2_image = cv2.resize(cv2_gray_image, (100, 100))
cv2_image = np.array(cv2_image).reshape(-1, 100, 100, 1) # reshapes the image into accepted dimensions (4D tensor)
cv2_image = cv2_image.astype('float32') # change data type of image to float32
cv2_image = cv2_image / 255.0

## Sample CloudWatch Logger

### Get Current Timestamp

In [1]:
import time
from datetime import datetime
timestamp = int(time.time()*1000)
timestamp

1579705485858

### Create Log Stream within Notebook Log Group

In [17]:
client = boto3.client('logs')
response = client.create_log_stream(
    logGroupName='/aws/sagemaker/NotebookInstances',
    logStreamName='inference_debugger'
)

### Create Log event within Log Stream

In [19]:
response = client.put_log_events(
    logGroupName='/aws/sagemaker/NotebookInstances',
    logStreamName='inference_debugger',
    logEvents=[
        {
            'timestamp': timestamp,
            'message': np.array2string(pillow_image, floatmode='unique')
        },
    ]
)

### Use sequence token obtained from response to continue adding more events

In [20]:
token = response['nextSequenceToken']

response = client.put_log_events(
    logGroupName='/aws/sagemaker/NotebookInstances',
    logStreamName='inference_debugger',
    logEvents=[
        {
            'timestamp': timestamp,
            'message': np.array2string(pillow_image, floatmode='unique')
        },
    ],
    sequenceToken=token
)